In [3]:
!pip install easyocr

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.9 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 46.5 MB/s eta 0:00:00


In [49]:
!pip install --upgrade python-telegram-bot

   ---------------------------------------- 0.0/717.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/717.1 kB ? eta -:--:--
   ---------------------------------------- 717.1/717.1 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: python-telegram-bot
    Found existing installation: python-telegram-bot 20.3
    Uninstalling python-telegram-bot-20.3:
      Successfully uninstalled python-telegram-bot-20.3


In [51]:
!pip install --upgrade httpx

In [55]:
!pip show python-telegram-bot httpx jupyterlab

Name: python-telegram-bot
Version: 20.3
Summary: We have made you a wrapper you can't refuse
Home-page: https://python-telegram-bot.org/
Author: Leandro Toledo
Author-email: devs@python-telegram-bot.org
License: LGPLv3
Location: D:\Anaconda\Lib\site-packages
Requires: httpx
Required-by: 
---
Name: httpx
Version: 0.24.1
Summary: The next generation HTTP client.
Home-page: https://github.com/encode/httpx
Author: 
Author-email: Tom Christie <tom@tomchristie.com>
License: 
Location: D:\Anaconda\Lib\site-packages
Requires: certifi, httpcore, idna, sniffio
Required-by: aext-shared, gradio, gradio_client, jupyterlab, python-telegram-bot, safehttpx
---
Name: jupyterlab
Version: 4.3.4
Summary: JupyterLab computational environment
Home-page: https://jupyter.org
Author: 
Author-email: Jupyter Development Team <jupyter@googlegroups.com>
License: Copyright (c) 2015-2024 Project Jupyter Contributors
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification

In [57]:
!pip install python-telegram-bot==20.3

In [1]:
!nvidia-smi

Sun Aug 17 13:16:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.97                 Driver Version: 580.97         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:06:00.0  On |                  N/A |
|  0%   47C    P8             14W /  170W |     467MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from PIL import Image
import cv2
import numpy as np
import easyocr

D:\Anaconda\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
D:\Anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
D:\Anaconda\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [9]:
# Пути к датасету
dataset_dir = os.path.expanduser("~/Desktop/datasets/GTSRB")
output_dir = os.path.expanduser("~/Desktop/datasets/yolo_gtsrb")
test_images_dir = "C:/Users/user/Desktop/test_images"
SAVE_DIR = "./detected_signs"
os.makedirs(SAVE_DIR, exist_ok=True)

In [9]:
# Создаем папки
# os.makedirs(f"{output_dir}/images/train", exist_ok=True)
# os.makedirs(f"{output_dir}/images/val", exist_ok=True)
# os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
# os.makedirs(f"{output_dir}/labels/val", exist_ok=True)

In [11]:
csv_path = os.path.join(dataset_dir, "Train.csv")  # или как называется у тебя
df_all = pd.read_csv(csv_path, sep=';')  # sep=';' если CSV с ;, иначе sep=','

In [13]:
csv_path = "~/Desktop/datasets/gtsrb/Train.csv"
df_all = pd.read_csv(csv_path, sep=',')  # sep=',' для твоего случая
df_all.columns = df_all.columns.str.strip()  # убираем лишние пробелы
print(df_all.columns.tolist())  # теперь должно быть корректно

['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId', 'Path']


In [15]:
df_all.columns = df_all.columns.str.strip()

In [31]:
# Функция конвертации координат в YOLO формат
def convert_to_yolo(row, img_width, img_height):
    x_center = ((row['Roi.X1'] + row['Roi.X2']) / 2) / img_width
    y_center = ((row['Roi.Y1'] + row['Roi.Y2']) / 2) / img_height
    width = (row['Roi.X2'] - row['Roi.X1']) / img_width
    height = (row['Roi.Y2'] - row['Roi.Y1']) / img_height
    class_id = int(row['ClassId'])
    return f"{class_id} {x_center} {y_center} {width} {height}"

In [61]:
class_counts = df_all['ClassId'].value_counts()
print(class_counts)

ClassId
2     2250
1     2220
13    2160
12    2100
38    2070
10    2010
4     1980
5     1860
25    1500
9     1470
7     1440
3     1410
8     1410
11    1320
18    1200
35    1200
17    1110
14     780
31     780
33     689
15     630
26     600
28     540
23     510
30     450
16     420
6      420
34     420
22     390
36     390
40     360
20     360
21     330
39     300
29     270
24     270
32     240
27     240
41     240
42     240
37     210
0      210
19     210
Name: count, dtype: int64


In [33]:
# Делим на train и val (например 80/20)
df_train, df_val = train_test_split(df_all, test_size=0.2, random_state=42, stratify=df_all['ClassId'])

In [35]:
# Функция подготовки данных
def prepare_dataset(df, split):
    for idx, row in df.iterrows():
        img_path = os.path.join(dataset_dir, row['Path'])
        if not os.path.exists(img_path):
            print(f"Пропущено: {img_path}")
            continue

        # Копируем изображение
        shutil.copy(img_path, f"{output_dir}/images/{split}/{os.path.basename(row['Path'])}")

        # Получаем размеры картинки
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Создаём YOLO аннотацию
        yolo_annotation = convert_to_yolo(row, img_width, img_height)
        txt_name = os.path.splitext(os.path.basename(row['Path']))[0] + ".txt"
        with open(os.path.join(output_dir, "labels", split, txt_name), 'w') as f:
            f.write(yolo_annotation + "\n")

In [41]:
# Подготавливаем train и val
prepare_dataset(df_train, "train")
prepare_dataset(df_val, "val")

In [67]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
NVIDIA GeForce RTX 3060


In [69]:
# Загружаем модель (yolov8n – маленькая, можно заменить на yolov8s/m/l/x)
model = YOLO("yolov8n.pt")  

# Параметры обучения
train_params = {
    "data": os.path.join(output_dir, "gtsrb.yaml"),
    "epochs": 40,
    "batch": 16,
    "imgsz": 640,
    "device": 0,        # если есть GPU, иначе 'cpu'
    "patience": 5,      # ранняя остановка при отсутствии улучшений
    "name": "gtsrb_yolov8",
    "augment": True    # False – без аугментации, True – с аугментацией
}

# Запуск обучения
model.train(**train_params)

New https://pypi.org/project/ultralytics/8.3.185 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\user/Desktop/datasets/yolo_gtsrb\gtsrb.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=gtsrb_yolov85, nbs=

train: Scanning C:\Users\user\Desktop\datasets\yolo_gtsrb\labels\train... 31993 images, 0 backgrounds, 0 corrupt: 100%|


train: New cache created: C:\Users\user\Desktop\datasets\yolo_gtsrb\labels\train.cache
val: Fast image access  (ping: 0.20.1 ms, read: 5.03.0 MB/s, size: 2.4 KB)


val: Scanning C:\Users\user\Desktop\datasets\yolo_gtsrb\labels\val.cache... 7842 images, 0 backgrounds, 0 corrupt: 100%


Plotting labels to runs\detect\gtsrb_yolov85\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\gtsrb_yolov85
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       7842       7842      0.996      0.998      0.995      0.973



40 epochs completed in 3.408 hours.
Optimizer stripped from runs\detect\gtsrb_yolov85\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\gtsrb_yolov85\weights\best.pt, 6.2MB

Validating runs\detect\gtsrb_yolov85\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 72 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 246/246 [01:


                   all       7842       7842      0.998      0.997      0.995      0.973
        speed_limit_20         42         42      0.996          1      0.995      0.976
        speed_limit_30        444        444          1          1      0.995      0.972
        speed_limit_50        450        450          1      0.998      0.995      0.968
        speed_limit_60        282        282          1          1      0.995      0.972
        speed_limit_70        396        396      0.995          1      0.995      0.975
        speed_limit_80        372        372      0.999          1      0.995      0.967
   restriction_ends_80         84         84      0.998          1      0.995      0.949
       speed_limit_100        288        288          1          1      0.995      0.978
       speed_limit_120        282        282      0.999          1      0.995      0.972
         no_overtaking        294        294      0.999          1      0.995       0.97
  no_overtaking_truck

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001D35F267B00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    

In [71]:
model = YOLO("C:\\Users\\user\\runs\\detect\\gtsrb_yolov85\\weights\\best.pt")

In [77]:
# Запускаем предсказания на папке с изображениями
results = model.predict(
    source="C:/Users/user/Desktop/test_images",  # папка с картинками
    save=True,           # сохранить результаты
    save_txt=True,       # сохранить координаты боксов в txt
    conf=0.5,            # порог уверенности
    device=0             # GPU (если доступно)
)


image 1/1 C:\Users\user\Desktop\test_images\natsionalnyy_proekt_rossii_sozdanie_kachestvennykh_i_bezopasnykh_dorog_idn500_blog.jpg: 448x640 (no detections), 93.5ms
Speed: 12.7ms preprocess, 93.5ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict2
15 labels saved to runs\detect\predict2\labels


In [81]:
# Путь к твоему датасету
labels_root = r"C:/Users/user/Desktop/datasets/yolo_gtsrb_segment/labels"

# Проходим по train и val
for split in ["train", "val"]:
    labels_dir = os.path.join(labels_root, split)
    for fname in os.listdir(labels_dir):
        if not fname.endswith(".txt"):
            continue

        file_path = os.path.join(labels_dir, fname)

        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # на всякий случай пропустим кривые строки
            parts[0] = "0"  # заменяем класс на 0
            new_lines.append(" ".join(parts))

        with open(file_path, "w") as f:
            f.write("\n".join(new_lines))

In [ ]:
model = YOLO("yolov8n.pt")  # можно n/s/m в зависимости от мощности
model.train(
    data="C:/Users/user/Desktop/datasets/yolo_gtsrb_segment/signs.yaml",
    epochs=40,
    imgsz=640,
    batch=16,
    name="sign_detector_single_class"
)

New https://pypi.org/project/ultralytics/8.3.186 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/user/Desktop/datasets/yolo_gtsrb_segment/signs.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=sign_de

train: Scanning C:\Users\user\Desktop\datasets\yolo_gtsrb_segment\labels\train... 31993 images, 0 backgrounds, 0 corrup


train: New cache created: C:\Users\user\Desktop\datasets\yolo_gtsrb_segment\labels\train.cache
val: Fast image access  (ping: 0.20.1 ms, read: 0.20.1 MB/s, size: 2.4 KB)


val: Scanning C:\Users\user\Desktop\datasets\yolo_gtsrb_segment\labels\val... 7842 images, 0 backgrounds, 0 corrupt: 10


val: New cache created: C:\Users\user\Desktop\datasets\yolo_gtsrb_segment\labels\val.cache
Plotting labels to runs\detect\sign_detector_single_class2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\sign_detector_single_class2
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      24/40      2.86G     0.3602     0.2015      1.009         19        640: 100%|██████████| 2000/2000 [04:14<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 246/246 [00:


                   all       7842       7842          1          1      0.995      0.974

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      25/40      2.86G     0.3543     0.1974      1.007         30        640: 100%|██████████| 2000/2000 [04:13<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 246/246 [00:


                   all       7842       7842          1          1      0.995      0.973

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      26/40      2.86G     0.3491     0.1948      1.004         36        640:  45%|████▌     | 902/2000 [01:54<02:26, 

In [5]:
!yolo detect train resume model=runs/detect/sign_detector_single_class2/weights/last.pt verbose=True

New https://pypi.org/project/ultralytics/8.3.186 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/user/Desktop/datasets/yolo_gtsrb_segment/signs.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\detect\sign_detector_single_class2\weights\last.pt, momentum=0.937

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



New https://pypi.org/project/ultralytics/8.3.186 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/user/Desktop/datasets/yolo_gtsrb_segment/signs.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\detect\sign_detector_single_class2\weights\last.pt, momentum=0.937

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

